# Inventory Reorder Planner — Notebook Template
Upload `final_orders.csv` and `Config_Template__fill_on_hand__lead_time_days__MOQ_.csv` next to this notebook.


In [ ]:
%pip install -q pandas numpy openpyxl matplotlib
import pandas as pd, numpy as np, math
from pathlib import Path
ORDERS_FILE = Path('final_orders.csv')
CONFIG_FILE = Path('Config_Template__fill_on_hand__lead_time_days__MOQ_.csv')
OUTPUT_XLSX = Path('reorder_recommendations_notebook.xlsx')


In [ ]:
import re
def norm_cols(df):
    return df.rename(columns={c: re.sub(r'[^a-z0-9]+','_',str(c).strip().lower()).strip('_') for c in df.columns})
def pick(df, names):
    for n in names:
        if n in df.columns: return n
    return None
def z_from_service_level(p):
    lookup = {0.80:0.84,0.85:1.04,0.90:1.28,0.95:1.64,0.97:1.88,0.98:2.05,0.99:2.33}
    keys = sorted(lookup)
    return lookup[min(keys, key=lambda k: abs(k - float(p)))]


In [ ]:
raw = pd.read_csv(ORDERS_FILE, dtype=str)
orders = norm_cols(raw)
date_col = pick(orders,['order_date','order_date_iso','date'])
sku_col  = pick(orders,['sku','product_sku','item_sku','product_name'])
qty_col  = pick(orders,['qty','quantity','units','order_qty'])
assert date_col and sku_col and qty_col, 'Missing date/sku/qty'
orders_df = pd.DataFrame({
    'order_date': pd.to_datetime(orders[date_col], errors='coerce'),
    'sku': orders[sku_col].astype(str).str.strip(),
    'qty': pd.to_numeric(orders[qty_col], errors='coerce')
})
orders_df = orders_df[orders_df['order_date'].notna() & orders_df['sku'].ne('') & orders_df['qty'].fillna(0).gt(0)].copy()
orders_df.head()


In [ ]:
daily = (orders_df.assign(order_date=orders_df['order_date'].dt.date)
         .groupby(['order_date','sku'], as_index=False)['qty'].sum()
         .sort_values(['order_date','sku']))
if not daily.empty:
    last_ts = pd.to_datetime(orders_df['order_date']).max()
    start_28 = (last_ts - pd.Timedelta(days=27)).date()
    daily_28 = daily[daily['order_date'] >= start_28]
    stats = (daily_28.groupby('sku')['qty']
             .agg(mean_daily_28d='mean', std_daily_28d=lambda s: float(np.std(s, ddof=1)) if len(s)>1 else 0.0)
             .reset_index().round(2))
else:
    stats = pd.DataFrame(columns=['sku','mean_daily_28d','std_daily_28d'])
daily.head(), stats.head()


In [ ]:
cfg_raw = pd.read_csv(CONFIG_FILE, dtype=str)
cfg = norm_cols(cfg_raw)
config_df = pd.DataFrame({
    'sku': cfg.get(pick(cfg,['sku','product_sku','item_sku','product_name','product']), pd.Series([],dtype=str)).astype(str).str.strip(),
    'on_hand': pd.to_numeric(cfg.get(pick(cfg,['on_hand','onhand','stock','qty_on_hand']),0), errors='coerce').fillna(0),
    'lead_time_days': pd.to_numeric(cfg.get(pick(cfg,['lead_time_days','lead_time','leadtime_days']),7), errors='coerce').fillna(7),
    'target_service_level': pd.to_numeric(cfg.get(pick(cfg,['target_service_level','service_level','sl']),0.90), errors='coerce').fillna(0.90),
    'MOQ': pd.to_numeric(cfg.get(pick(cfg,['moq','min_order_qty','minimum_order_quantity']),0), errors='coerce').fillna(0).astype(int)
})
config_df['case_pack'] = pd.to_numeric(cfg.get(pick(cfg,['case_pack','casepack','pack_size']),0), errors='coerce').fillna(0).astype(int)
config_df.head()


In [ ]:
HORIZON_DAYS=14
merged = config_df.merge(stats, on='sku', how='left').fillna({'mean_daily_28d':0.0,'std_daily_28d':0.0})
rows=[]
for _, r in merged.iterrows():
    z = z_from_service_level(float(r['target_service_level']))
    on_hand=float(r['on_hand']); mean=float(r['mean_daily_28d']); std=float(r['std_daily_28d']); L=int(r['lead_time_days'])
    safety = z * std * (L if L>0 else 1)**0.5
    rop = mean * L + safety
    target = mean * (L + HORIZON_DAYS) + safety
    needed = int(max(0, math.ceil(target - on_hand)))
    moq=int(r['MOQ']); cp=int(r.get('case_pack',0))
    if needed>0 and needed<moq: needed=moq
    if cp and needed>0: needed = cp * math.ceil(needed/cp)
    rows.append({'sku':r['sku'],'on_hand':on_hand,'mean_daily_28d':mean,'std_daily_28d':std,'lead_time_days':L,
                 'service_level':float(r['target_service_level']),'z_score_used':z,'safety_stock':round(safety,2),
                 'reorder_point':round(rop,2),'target_stock(lead+horizon)':round(target,2),'recommended_order_qty':needed,
                 'MOQ':moq,'case_pack':cp,'status':'REORDER' if (on_hand<=rop and needed>0) else 'OK'})
reorder_df = pd.DataFrame(rows).sort_values(['status','sku'], ascending=[False,True])
reorder_df.head()


In [ ]:
with pd.ExcelWriter(OUTPUT_XLSX, engine='openpyxl') as w:
    reorder_df.to_excel(w, index=False, sheet_name='Reorder')
    daily.to_excel(w, index=False, sheet_name='DailyDemand')
    orders_df.to_excel(w, index=False, sheet_name='CleanOrders')
    config_df.to_excel(w, index=False, sheet_name='ConfigUsed')
print('Saved:', OUTPUT_XLSX.resolve())
